In [ ]:
import requests
import pandas as pd
import json
import tensorflow as tf
import base64

In [ ]:
data = pd.read_csv("./data/Obesity Classification.csv")


In [ ]:
data

In [ ]:
data["Label"].unique()

In [ ]:
print(requests.get("https://obesityclassification-production.up.railway.app/v1/models/cc-model").json())


In [ ]:
def prepare_json(instance_dict):
    feature_spec = {}

    for key, value in instance_dict.items():
        if key == "Age":
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[int(value)]))
        elif isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]))
        else:
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[float(value)]))

    example = tf.train.Example(features=tf.train.Features(feature=feature_spec)).SerializeToString()
    serialized_b64 = base64.b64encode(example).decode("utf-8")

    return json.dumps({"instances": [serialized_b64]})

# Data instance
instance = {
    "Age": 25,
    "Gender": "Male",
    "Height": 175.0,
    "Weight": 80.0,
    "BMI": 25.3
}

# Kirim request
response = requests.post(
    "https://obesityclassification-production.up.railway.app/v1/models/cc-model:predict",
    headers={"Content-Type": "application/json"},
    data=prepare_json(instance),
)

print(response.status_code)
print(response.json())